In [1]:
%pip install --upgrade pip
%pip install google
%pip install --upgrade google-cloud-bigquery
%pip install --upgrade google-cloud-storage
%pip install --upgrade google-cloud-bigquery-storage
%pip install pandas
%pip install db-dtypes

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.cloud import bigquery
from google.cloud import storage
from google.oauth2 import service_account
import pandas as pd

# Clients


In [3]:
project_id = 'bigquery-midterm-exam'
dataset_id = 'datamodel_midterm_dataset'

In [4]:
# Initialize BigQuery client
credentials = service_account.Credentials.from_service_account_file(
    'bigquery-midterm-exam-5265cdf05f9b.json')

bigquery_client = bigquery.Client(project=project_id, credentials=credentials)
storage_client = storage.Client(project=project_id, credentials=credentials)

In [43]:
bucket_name = "datamodel_midterm"

In [44]:
customer_logs_path = 'customer_logs.csv'
ad_performance_path = 'ad_performance.csv'

In [46]:
# 上傳 customer_logs.csv 至 Google Cloud Storage
csv_file_path = 'customer_logs.csv'
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob('customer_logs.csv')
blob.upload_from_filename(csv_file_path)

# 將 CSV 資料作為外部表格載入 BigQuery
table_id = 'customer_logs'
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True
)
uri = f'gs://{bucket_name}/customer_logs.csv'

load_job = bigquery_client.load_table_from_uri(
    uri, f"{dataset_id}.{table_id}", job_config=job_config
)
load_job.result()  # 等待作業完成
print("上傳完成")

# 上傳 customer_logs.csv 至 Google Cloud Storage
csv_file_path = 'ad_performance.csv'
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob('ad_performance_logs.csv')
blob.upload_from_filename(csv_file_path)

# 將 CSV 資料作為外部表格載入 BigQuery
table_id = 'ad_performance_logs'
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True
)
uri = f'gs://{bucket_name}/ad_performance_logs.csv'

load_job = bigquery_client.load_table_from_uri(
    uri, f"{dataset_id}.{table_id}", job_config=job_config
)
load_job.result()  # 等待作業完成
print("上傳完成")

上傳完成
上傳完成


# Task 1


## 1.1 資料擷取與上傳


In [5]:
dataset_id = 'datamodel_midterm_dataset'
bucket_name = 'datamodel_midterm'

In [14]:
bucket = storage_client.bucket(bucket_name)

In [16]:
# Upload customer_logs.csv
customer_logs_blob = bucket.blob('customer_logs.csv')
customer_logs_blob.upload_from_filename('customer_logs.csv')

# Upload ad_performance_logs.csv
ad_performance_logs_blob = bucket.blob('ad_performance_logs.csv')
ad_performance_logs_blob.upload_from_filename('ad_performance.csv')

## 1.2 Transform Data


In [6]:
# Normalize 'score' in Google Trends
normalize_query = f"""
CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.normalized_international_top_rising_terms` AS
SELECT
  country_code,
  country_name,
  term,
  (score - MIN(score) OVER()) / (MAX(score) OVER() - MIN(score) OVER()) AS score
FROM
  `bigquery-public-data.google_trends.international_top_rising_terms` 
WHERE score IS NOT null
"""
job = bigquery_client.query(normalize_query)
job.result()
print("Score normalization completed.")


Score normalization completed.


## 1.3 Integrate Datasets


In [7]:
# Normalize 'score' in Google Trends
integrated_query = f"""
CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.combined_geotargets_criteria` AS
SELECT
    g.criteria_id,
    g.en_name AS geographic_target_name,
    g.country_code,
    m.target_city
FROM
    `bigquery-public-data.google_ads.geotargets` AS g
JOIN
    `bigquery-public-data.google_ads_geo_mapping_us.ads_geo_criteria_mapping` AS m
ON
    CAST(g.criteria_id AS INT64) = m.ads_criteria_id;
"""
job = bigquery_client.query(integrated_query)
job.result()
print("Score normalization completed.")


Score normalization completed.


In [8]:
# Normalize 'score' in Google Trends
integrated_query = f"""
CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.integrated_data` AS
SELECT trends.country_code, trends.country_name, trends.term, trends.score,
       ads.ad_id, ads.impressions, ads.clicks, ads.conversions,
       logs.user_action, analytics.totals.pageviews, analytics.totals.transactions
FROM `bigquery-midterm-exam.datamodel_midterm_dataset.normalized_international_top_rising_terms` AS trends
JOIN `bigquery-midterm-exam.datamodel_midterm_dataset.customer_logs` AS logs
ON trends.country_name = logs.country
JOIN `bigquery-midterm-exam.datamodel_midterm_dataset.ad_performance_logs` AS ads
ON trends.country_name = ads.country
JOIN `bigquery-public-data.google_analytics_sample.ga_sessions_20170801` AS analytics
ON trends.country_name = analytics.geoNetwork.country
JOIN `bigquery-midterm-exam.datamodel_midterm_dataset.combined_geotargets_criteria` AS geo
ON  trends.country_code = geo.country_code
WHERE RAND() < 0.001
"""
job = bigquery_client.query(integrated_query)
job.result()
print("integrated_query completed.")


integrated_query completed.
